In [1]:
print("OK!")

OK!


In [2]:
import os
os.chdir("../")

In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
#from pinecone.grpc import PineconeGRPC as Pinecone
#from pinecone import ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("data/")

In [7]:
# extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 212


In [11]:
# text_chunks

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
import huggingface_hub
import sentence_transformers

embeddings = download_hugging_face_embeddings()

c:\ProgramData\Anaconda3\envs\scholar\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\atq765\AppData\Local\Temp\ipykernel_39508\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [17]:
# query_result

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


index_name = "scholar-chatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [17]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [19]:
docsearch

In [20]:
import os
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
index_name = "scholar-chatbot" 
from langchain.vectorstores import Pinecone as LangchainPinecone
query = "Does extra HER-2/neu make breast cancer worse?"
docs = docsearch.similarity_search(query, k=3) 
print("Result:", docs)

Result: [Document(id='69470603-418e-469a-b6f8-0aa653ad63d5', metadata={'creationdate': '2007-01-15T15:29:22+00:00', 'creator': 'PyPDF', 'moddate': '2008-03-03T11:28:12-08:00', 'page': 1.0, 'page_label': '2', 'producer': 'Adobe PDF Library 7.0', 'rgid': 'PB:19364043_AS:101511233277955@1401213572478', 'source': 'data\\SlamonetalSCIENCE1987.pdf', 'total_pages': 8.0}, page_content='DOI: 10.1126/science.3798106 \n, 177 (1987); 235Science\n  et al.DJ Slamon,\noncogene\nsurvival with amplification of the HER-2/neu \nHuman breast cancer: correlation of relapse and'), Document(id='fe4a9ba7-23b2-4155-bbce-a1783fb7591e', metadata={'creationdate': '2007-01-15T15:29:22+00:00', 'creator': 'PyPDF', 'moddate': '2008-03-03T11:28:12-08:00', 'page': 0.0, 'page_label': '1', 'producer': 'Adobe PDF Library 7.0', 'rgid': 'PB:19364043_AS:101511233277955@1401213572478', 'source': 'data\\SlamonetalSCIENCE1987.pdf', 'total_pages': 8.0}, page_content='cancer: correlation of relapse and survival with ampliﬁcation 

In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [26]:
from langchain.llms import CTransformers
#from langchain.llms import CTransformers

llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q8_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':256,
                          'temperature':0.01})

In [43]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [44]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])